# Baseline Methods for OOD Digit Classification

- Decision Tree
- Random Forest
- Regular MLP
- AdaBoost
- SVM

## Data Preprocessing

Think about the following

- Should data have 3 channels or grayscale (1 channel)
- Should we use a scaler to center mean and scale to unit variance

In [1]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
from typing import Union, List
import matplotlib.pyplot as plt
from DGDataset import DGDataset
from collections import OrderedDict
from torch.utils.data import DataLoader

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, mean_squared_error, classification_report, confusion_matrix, precision_recall_curve, PrecisionRecallDisplay, RocCurveDisplay
from joblib import dump, load

/home/huakun/anaconda3/envs/AI/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sns.set_style('darkgrid')

## Helper Functions

In [3]:
datasets = ['mnist', 'mnist_m', 'svhn', 'syn']
target_domain = 'syn'
scaler = StandardScaler()

In [4]:
def get_performance_metrics(predictions: np.ndarray, labels: np.ndarray):
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    mse = mean_squared_error(labels, predictions)
    cm = confusion_matrix(labels, predictions)
    classification_rpt = classification_report(labels, predictions, output_dict=True)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "mse": mse,
        "cm": cm,
        "classification_rpt": classification_rpt,
        "classification_rpt_df": pd.DataFrame(classification_rpt).transpose()
    }
# predictions = svm_grid.predict(test_data)
# performance = get_performance_metrics(predictions, test_labels)


In [5]:
def load_dataset(datasets: List[str], target_domain='syn', mode: str='train'):
    # datasets_ = datasets.copy()
    # datasets_.remove(target_domain)
    dataset = DGDataset(datasets, mode=mode)
    dataloader = DataLoader(dataset, batch_size=100)
    data, labels, domains = [], [], []
    for d, label, domain in dataloader:
        data.extend(d.numpy())
        labels.extend(label.numpy())
        domains.extend(domain.numpy())
    data = np.array(data)
    if len(data.shape) == 4:
        # has a color channel dimension
        data = data.reshape(len(data), np.prod(data.shape[1:])) # flatten each image to a vector
    return data, labels, domains

In [6]:
train_datasets = datasets.copy()
train_datasets.remove(target_domain)
train_data, train_labels, train_domains = load_dataset(train_datasets, mode='train')
val_data, val_labels, val_domains = load_dataset(train_datasets, mode='val')
test_data, test_labels, test_domains = load_dataset([target_domain], mode='test')

## SVM

In [7]:
parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': (1, 10),
    'gamma': ('scale', 'auto'),
    'decision_function_shape': ('ovo', 'ovr')
}
svm_grid = GridSearchCV(svm.SVC(), parameters).fit(train_data, train_labels)
print("Best SVM Parameters")
for k, v in svm_grid.best_params_.items():
    print(f"\t{k}: {v}")
dump(svm_grid, './models/svm_grid.joblib')
svm_model = SVC(C=svm_grid.best_params_['C'], 
                kernel=svm_grid.best_params_['kernel'], 
                gamma=svm_grid.best_params_['gamma'], 
                decision_function_shape=svm_grid.best_params_['decision_function_shape']).fit(train_data, train_labels)
dump(svm_grid, './models/best_svm_model.joblib')


Best SVM Parameters
	C: 10
	decision_function_shape: ovo
	gamma: scale
	kernel: rbf


['./models/best_svm_model.joblib']

In [8]:
print(f"Accuracy: {round(accuracy_score(svm_model.predict(test_data), test_labels) * 100, 2)}%")

Accuracy: 55.37%


In [9]:
predictions = svm_model.predict(test_data)
svm_performance = get_performance_metrics(predictions, test_labels)
for k in ['accuracy', 'precision', 'recall', 'mse', 'cm']:
    print(f'{k}:', '\n', svm_performance[k], '\n')

svm_performance['classification_rpt_df']

accuracy: 
 0.5536666666666666 

precision: 
 0.5585653726812658 

recall: 
 0.5536666666666666 

mse: 
 8.566166666666666 

cm: 
 [[352  27  16  20  38  18  50  37  27  15]
 [ 17 324  43  42  31  12  18  81  23   9]
 [ 23  30 382  22  24  17   8  62  25   7]
 [ 28  46  53 281  27  53  11  41  20  40]
 [ 11  33  23  11 445   3  24  15  13  22]
 [ 38  22  24  45  25 350  25  33  15  23]
 [113   6  14   5  45  62 299  25  20  11]
 [ 16  72  76  12   9  20  17 354  21   3]
 [ 51  10  22  10  49  78  84  18 250  28]
 [ 58  18  27   8  47  47  15  54  41 285]] 



,precision,recall,f1-score,support
0,0.497878,0.586667,0.538638,600.000000
1,0.551020,0.540000,0.545455,600.000000
2,0.561765,0.636667,0.596875,600.000000
3,0.616228,0.468333,0.532197,600.000000
4,0.601351,0.741667,0.664179,600.000000
5,0.530303,0.583333,0.555556,600.000000
6,0.542650,0.498333,0.519548,600.000000
7,0.491667,0.590000,0.536364,600.000000
8,0.549451,0.416667,0.473934,600.000000
9,0.643341,0.475000,0.546500,600.000000


## Decision Tree

In [10]:
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(train_data, train_labels)
print(f"Accuracy: {round(accuracy_score(dt_clf.predict(test_data), test_labels) * 100, 2)}%")

Accuracy: 20.57%


## Random Forest

In [11]:
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(train_data, train_labels)
print(f"Accuracy: {round(accuracy_score(rf_clf.predict(test_data), test_labels) * 100, 2)}%")

Accuracy: 9.27%


## MLP

In [12]:
parameters = {
    'activation': ('identity', 'logistic', 'tanh', 'relu'),
    'solver': ('lbfgs', 'sgd', 'adam'),
    'learning_rate': ('constant', 'invscaling', 'adaptive')
}
mlp_grid = GridSearchCV(MLPClassifier(shuffle=True), parameters).fit(train_data, train_labels)
print(f"Accuracy: {round(accuracy_score(mlp_grid.predict(test_data), test_labels) * 100, 2)}%")
dump(mlp_grid, './models/mlp_grid.joblib')
mlp_clf = MLPClassifier(
    shuffle=True,
    activation=mlp_grid.best_params_['activation'],
    solver=mlp_grid.best_params_['solver'],
    learning_rate=mlp_grid.best_params_['learning_rate']).fit(train_data, train_labels)
dump(mlp_clf, './models/best_mlp.joblib')

/home/huakun/anaconda3/envs/AI/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


KeyboardInterrupt: 

In [15]:
mlp_clf = MLPClassifier(
    shuffle=True,
    activation='relu',
    solver='adam').fit(train_data, train_labels)
dump(mlp_clf, './models/best_mlp.joblib')
predictions = mlp_clf.predict(test_data)
mlp_performance = get_performance_metrics(predictions, test_labels)
for k in ['accuracy', 'precision', 'recall', 'mse', 'cm']:
    print(f'{k}:', '\n', mlp_performance[k], '\n')

mlp_performance['classification_rpt_df']

accuracy: 
 0.6415 

precision: 
 0.6424267600122133 

recall: 
 0.6415 

mse: 
 7.132833333333333 

cm: 
 [[385  26   6  33  32   3  31  33  24  27]
 [ 10 378  24  35  19   6  16  82  20  10]
 [  9  15 431  22  23   8  12  48  14  18]
 [ 27  18  23 395  19  37  13  32  12  24]
 [ 13  29  13   9 450   2  21  12  15  36]
 [ 18  17  19  46  12 392  16  25  14  41]
 [ 72   4   9  10  32  77 345   9  32  10]
 [  6  55  58   7  11   9  11 416  17  10]
 [ 35  13  15  27  34  54  56  15 309  42]
 [ 35  17  18  29  62  22  12  38  19 348]] 



,precision,recall,f1-score,support
0,0.631148,0.641667,0.636364,600.0000
1,0.660839,0.630000,0.645051,600.0000
2,0.699675,0.718333,0.708882,600.0000
3,0.644372,0.658333,0.651278,600.0000
4,0.648415,0.750000,0.695518,600.0000
5,0.642623,0.653333,0.647934,600.0000
6,0.647280,0.575000,0.609003,600.0000
7,0.585915,0.693333,0.635115,600.0000
8,0.649160,0.515000,0.574349,600.0000
9,0.614841,0.580000,0.596913,600.0000


## AdaBoost

In [13]:
adaboost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
adaboost_clf.fit(train_data, train_labels)
print(f"Accuracy: {round(accuracy_score(adaboost_clf.predict(test_data), test_labels) * 100, 2)}%")

Accuracy: 11.12%
